In [1]:
###################
#   Ex00: Setup   #
###################

import psycopg2
import os
import getpass
from utils.utils import run_sql, run_task

if os.uname()[0] == "Linux":
    USER = "postgres"
else:
    USER = getpass.getuser()

#################
#   FUNCTIONS   #
#################

@run_task("'{0}' database creation")
def create_database(db: str):
    run_sql("CREATE DATABASE {}".format(db),
            user=USER, db="postgres", commit=True)

@run_task("'{0}' database deletion")
def drop_database(db: str):
    run_sql("DROP DATABASE IF EXISTS {}".format(db),
            user=USER, db="postgres", commit=True)

@run_task("Password change for user '{0}'")   
def change_passwd(user="dd", passwd:str="dd"):
    run_sql("ALTER USER {} PASSWORD '{}';".format(user, passwd),
            user=USER, db="postgres", commit=True)

@run_task("'{0}' user creation")
def create_user(user: str, password: str):
    run_sql("CREATE USER {} LOGIN PASSWORD '{}'".format(user, password),
            user=USER, db="postgres", commit=True)

@run_task("'{0}' user deletion")
def drop_user(user: str):
    run_sql("DROP USER IF EXISTS {}".format(user),
            user=USER, db="postgres", commit=True)

@run_task("'{0}' database owner change to '{1}'")
def alter_database(db: str, user: str):
    run_sql("ALTER DATABASE {} OWNER TO {}".format(db, user),
            user=USER, db="postgres", commit=True)
    
def main():
    change_passwd(USER, "12345")
    drop_database("appstore_games")
    create_database("appstore_games")
    drop_user("postgres_user")
    create_user("postgres_user", "12345")
    alter_database("appstore_games", "postgres_user")


if __name__ == "__main__":
    main()

2020-02-22 15:08:40 :: INFO :: Password change for user 'fxbabin' Done !
2020-02-22 15:08:40 :: INFO :: 'appstore_games' database deletion Done !
2020-02-22 15:08:40 :: INFO :: 'appstore_games' database creation Done !
2020-02-22 15:08:40 :: INFO :: 'postgres_user' user deletion Done !
2020-02-22 15:08:40 :: INFO :: 'postgres_user' user creation Done !
2020-02-22 15:08:40 :: INFO :: 'appstore_games' database owner change to 'postgres_user' Done !


In [2]:
###################
#   Ex01: Clean   #
###################


import pandas as pd
import re


def df_nan_filter(df):
    """Apply filters on NaN values
    Args:
      df: pandas dataframe.
    Returns:
      Filtered Dataframe.
    Raises:
      This function shouldn't raise any Exception.
    """
    # remove "Size" row if value is null
    # set "EN" as langage if value is null
    # set 0.0 as price if value is null
    # set median as user_rating if value is null
    # replace nan counts with 1

    df = df[df["Size"].notnull()].copy()
    df["Languages"].fillna("EN", inplace=True)
    df["Price"].fillna(0.0, inplace=True)
    df["Average User Rating"].fillna(
            df["Average User Rating"].median(), inplace=True)
    df["User Rating Count"].fillna(1, inplace=True)
    df.drop_duplicates(subset="ID", inplace=True)
    return (df)


def string_filter(s: str):
    """Apply filters in order to clean the string.
    Args:
      s: string.
    Returns:
      Filtered String.
    Raises:
      This function shouldn't raise any Exception.
    """
    # filter : \\t, \\n, \\U1a1b2c3d4, \\u1a2b, \\x1a
    # turn \' into '
    # remove remaining \\
    # turn multiple spaces into one space
    s = re.sub(r'''\\+(t|n|U[a-z0-9]{8}|u[a-z0-9]{4}|x[a-z0-9]{2}|[\.]{2})''', ' ', s)
    s = s.replace('\\\'', '\'').replace('\\\\', '\\')
    s = re.sub(r' +', ' ', s)
    return (s)


def change_date_format(date: str):
    """Change date format from dd/mm/yy to yy-mm-dd
    Args:
      date: a string representing the date.
    Returns:
      The date in the format yy-mm-dd.
    Raises:
      This function shouldn't raise any Exception.
    """
    tmp = date.split('/')
    return (tmp[2]+"-"+tmp[1]+"-"+tmp[0])

@run_task("Cleaning csv", oneline=False)
def main():
    df = pd.read_csv("appstore_games.csv")

    # selecting columns
    df = (
        df[["ID", "Name", "Average User Rating",
            "User Rating Count", "Price", "Description",
            "Developer", "Age Rating", "Languages",
            "Size", "Primary Genre", "Genres",
            "Original Release Date", "Current Version Release Date"]]
    )
    # apply nan_filter
    df = df_nan_filter(df)
    #print(df.head(50))

    # apply string_filter
    df["Name"] = df["Name"].apply(lambda x: string_filter(x))
    df["Description"] = df["Description"].apply(lambda x: string_filter(x))

    # apply change_date_format
    df["Original Release Date"] = (
            df["Original Release Date"].apply(lambda x: change_date_format(x))
    )
    df["Current Version Release Date"] = (
        df["Current Version Release Date"].apply(
            lambda x: change_date_format(x)
        )
    )

    # apply int conversion
    df["Age Rating"] = df["Age Rating"].apply(lambda x: int(x[:-1]))
    df['User Rating Count'] = (
        df['User Rating Count'].apply(lambda x: int(float(x)))
    )
    df['Size'] = df['Size'].apply(lambda x: int(float(x)))

    # saving file as csv
    df.to_csv("appstore_games.cleaned.csv", index=False)


if __name__ == "__main__":
    main()

2020-02-22 15:08:41 :: INFO :: Cleaning csv started.
2020-02-22 15:08:43 :: INFO :: Cleaning csv Done !


In [3]:
#######################
#   Ex02: Normalize   #
#######################


import re
import pandas as pd


def nf_normalization_genres(df):
    """Get normalized genres dataframe.
    Args:
      df: pandas dataframe.
    Returns:
      df_genres: pandas dataframe.
    Raises:
      This function shouldn't raise any Exception.
    """
    df_genres = df[["ID", "Primary Genre", "Genres"]].copy()
    df_genres['Genre'] = df_genres['Genres'].apply(lambda x: re.sub(r' & ', '_&_', x))
    df_genres["Genre"] = df_genres["Genre"].apply(lambda x: re.split(r'[ ,]+', x))
    df_genres = df_genres.drop(columns=['Genres'])
    df_genres = df_genres.explode('Genre')
    df_genres = df_genres[df_genres["Genre"] != "Games"]
    df_genres = df_genres.reset_index(drop=True)
    return(df_genres)


def nf_normalization_languages(df):
    """Get normalized languages dataframe.
    Args:
      df: pandas dataframe.
    Returns:
      df_languages: pandas dataframe.
    Raises:
      This function shouldn't raise any Exception.
    """
    df_languages = df[["ID", "Languages"]].copy()
    df_languages["Language"] = (
      df_languages["Languages"].apply(lambda x: re.split(r'[ ,]+', x))
    )
    df_languages = df_languages.drop(columns=['Languages'])
    df_languages = df_languages.explode('Language')
    df_languages = df_languages.reset_index(drop=True)
    return(df_languages)

@run_task('Normalization of data', oneline=False)
def main():
    df = pd.read_csv("appstore_games.cleaned.csv")

    # normalize dataframes
    df_genres = nf_normalization_genres(df)
    df_languages = nf_normalization_languages(df)
    df = df.drop(['Languages', 'Primary Genre', 'Genres'], axis=1)

    # saving dataframes into csv files
    df.to_csv("appstore_games.normalized.csv", index=False)
    df_genres.to_csv("appstore_games_genres.normalized.csv", index=False)
    df_languages.to_csv("appstore_games_languages.normalized.csv", index=False)


if __name__ == "__main__":
    main()


2020-02-22 15:08:43 :: INFO :: Normalization of data started.
2020-02-22 15:08:44 :: INFO :: Normalization of data Done !


In [5]:
######################
#   Ex03: Populate   #
######################

import pandas as pd
import psycopg2
from utils.utils import run_sql, get_connection

#############
#   UTILS   #
#############


@run_task("Deleting table '{0}'")
def delete_table(table: str):
    run_sql("DROP TABLE IF EXISTS {};".format(table), commit=True)


def display_table(table: str):
    res = run_sql("SELECT * FROM {} LIMIT 10".format(table), fetch=True)
    for row in res:
        print(row)


@run_task("Importing csv '{0}'")
def import_csv(file):
    df = pd.read_csv(file)
    return (df)


##############################
#   CREATE TABLE FUNCTIONS   #
##############################


@run_task("Creating table '{0}'")
def create_table(table, query):
    run_sql(query.format(table), commit=True)


def create_table_appstore_games():
    create_table("appstore_games",
                 """
                 CREATE TABLE IF NOT EXISTS {} (
                     Game_Id bigint PRIMARY KEY,
                     Name varchar,
                     Avg_user_rating float,
                     User_rating_count int,
                     Price float,
                     Description varchar,
                     Developer varchar,
                     Age_rating int,
                     Size bigint,
                     Release_date date,
                     Last_update date
                 );""")


def create_table_appstore_games_languages():
    create_table("appstore_games_languages",
                 """
                 CREATE TABLE IF NOT EXISTS {} (
                     Id bigint PRIMARY KEY,
                     Game_Id bigint REFERENCES appstore_games(Game_Id),
                     Language varchar
                 );""")


def create_table_appstore_games_genres():
    create_table("appstore_games_genres",
                 """
                 CREATE TABLE IF NOT EXISTS {} (
                     Id bigint PRIMARY KEY,
                     Game_Id bigint REFERENCES appstore_games(Game_Id),
                     Primary_genre varchar,
                     Genre varchar
                 );""")


##########################
#   POPULATE FUNCTIONS   #
##########################


@run_task("Populating table '{0}'", oneline=False)
def populate_table(table, query, df, index=False):
    conn = get_connection()
    curr = conn.cursor()
    df_rows = list(df.itertuples(index=index))
    args_str = b",".join(curr.mogrify("%s", (row,)) for row in df_rows)
    curr.execute(query.format(table) + args_str.decode())
    conn.commit()
    conn.close()


def populate_appstore_games(df):
    populate_table("appstore_games",
                   """
                   INSERT INTO {}
                       (Game_Id,
                       Name,
                       Avg_user_rating,
                       User_rating_count,
                       Price,
                       Description,
                       Developer,
                       Age_rating,
                       Size,
                       Release_date,
                       Last_update) VALUES """, df)


def populate_appstore_games_genres(df_genres):
    populate_table("appstore_games_genres",
                   """
                   INSERT INTO {}
                       (Id,
                       Game_Id,
                       Primary_genre,
                       Genre) VALUES """, df_genres, index=True)


def populate_appstore_games_languages(df_languages):
    populate_table("appstore_games_languages",
                   """
                   INSERT INTO {}
                       (Id,
                       Game_Id,
                       Language) VALUES """, df_languages, index=True)


############
#   MAIN   #
############


def main():
    df = import_csv("appstore_games.normalized.csv")
    df_genres = import_csv("appstore_games_genres.normalized.csv")
    df_languages = import_csv("appstore_games_languages.normalized.csv")

    delete_table("appstore_games_genres")
    delete_table("appstore_games_languages")
    delete_table("appstore_games")

    create_table_appstore_games()
    create_table_appstore_games_genres()
    create_table_appstore_games_languages()

    populate_appstore_games(df)
    populate_appstore_games_genres(df_genres)
    populate_appstore_games_languages(df_languages)


if __name__ == "__main__":
    main()


2020-02-22 15:09:18 :: INFO :: Importing csv 'appstore_games.normalized.csv' Done !
2020-02-22 15:09:18 :: INFO :: Importing csv 'appstore_games_genres.normalized.csv' Done !
2020-02-22 15:09:18 :: INFO :: Importing csv 'appstore_games_languages.normalized.csv' Done !
2020-02-22 15:09:18 :: INFO :: Deleting table 'appstore_games_genres' Done !
2020-02-22 15:09:18 :: INFO :: Deleting table 'appstore_games_languages' Done !
2020-02-22 15:09:18 :: INFO :: Deleting table 'appstore_games' Done !
2020-02-22 15:09:18 :: INFO :: Creating table 'appstore_games' Done !
2020-02-22 15:09:18 :: INFO :: Creating table 'appstore_games_genres' Done !
2020-02-22 15:09:18 :: INFO :: Creating table 'appstore_games_languages' Done !
2020-02-22 15:09:18 :: INFO :: Populating table 'appstore_games' started.
2020-02-22 15:09:19 :: INFO :: Populating table 'appstore_games' Done !
2020-02-22 15:09:19 :: INFO :: Populating table 'appstore_games_genres' started.
2020-02-22 15:09:20 :: INFO :: Populating table 'a

In [6]:
####################
#   Ex04: Top100   #
####################

from utils.utils import run_sql

@run_task("Ex04 Top 100", oneline=False)
def get_top_100():
    res = run_sql("""
            SELECT Name, Avg_user_rating
                FROM appstore_games
                WHERE Name ~ '^[A-Za-z]'
                ORDER BY Avg_user_rating DESC, Name
                LIMIT 100
            """, fetch=True)
    for row in res:
        print(row[0])


def main():
    get_top_100()


if __name__ == "__main__":
    main()

2020-02-22 15:15:04 :: INFO :: Top 100 started.
2020-02-22 15:15:04 :: INFO :: Top 100 Done !


AFK Arena
APORIA
AbsoluteShell
Action Craft Mini Blockheads Match 3 Skins Survival Game
Adrift by Tack
Agadmator Chess Clock
Age Of Magic
Age of Giants: Tribal Warlords
Age of War Empires: Order Rise
Alicia Quatermain 2 (Platinum)
Alien Dog Evolution - Wild Stray Mutant Simulator
Alien Escape - Puzzle/Strategy
Alliance Wars: World Domination
AlterZone
AlternaMagic
Amaturufu
Ancient Empires Reloaded
Angry Drunks
Angry Zombie Tower Defense
Anomaly 2
Ant Factory !
Ant VS Candy
Anthill
Anti Chess: Classic Board Game
ArcadeDefense
Arizona Rose HD Full
Arkon: Chess Opening Explorer
Arroe - Archery with Balloons
Arrow Fire - Archery is Hard
Art Inc. - Collection Clicker
Art Of War 3:RTS Strategy Game
Art of Tactics: War Games
Astro Ant
Astroid Storm
Asymmetric
Atom Pop
Atoms GO!
Avadon 3: The Warborn HD
Avernum: Escape From the Pit HD
Avoid The Squares!
Baduk Cap
Bake Cake - Delicious Choice
Ball Crash! Jump Dots from Down
Ball Run!
Ball Slide
Balling Out
Banana Market
Bankrupt - Best Busines

In [ ]:
#######################
#   Ex05: Name_lang   #
#######################

from utils.utils import run_sql


@run_task("Ex05 Name_lang", oneline=False)
def get_name_lang():
    res = run_sql("""
            SELECT Name, Language
                FROM appstore_games
                INNER JOIN appstore_games_languages
                ON appstore_games.Game_Id =
                    appstore_games_languages.Game_Id
                WHERE appstore_games.Price > 5.0
                AND appstore_games.Price < 10.0
            """, fetch=True)
    for row in res:
        print(row[0], row[1])


def main():
    get_name_lang()


if __name__ == "__main__":
    main()

In [ ]:
#####################
#   Ex06: K-first   #
#####################

from utils.utils import run_sql


@run_task("Ex06 K-first", oneline=False)
def get_k_first():
    res = run_sql("""
                    SELECT Developer
                        FROM appstore_games
                        INNER JOIN appstore_games_genres
                        ON appstore_games.Game_Id = appstore_games_genres.Game_Id
                        WHERE Developer LIKE 'K%'
                        AND appstore_games_genres.Genre LIKE '%Casual%';
                 """, fetch=True)
    for row in res:
        print(row[0])


def main():
    get_k_first()


if __name__ == "__main__":
    main()

In [ ]:
#####################
#   Ex07: Seniors   #
#####################

from utils.utils import run_sql


@run_task("Ex07 Seniors", oneline=False)
def get_seniors():
    res = run_sql("""
                    SELECT Developer
                    FROM appstore_games
                    WHERE appstore_games.Release_date < '2008-08-01 00:00:00'
                    AND appstore_games.Last_update > '2018-01-01 00:00:00';
                 """, fetch=True)
    for row in res:
        print(row[0])


def main():
    get_seniors()


if __name__ == "__main__":
    main()

In [ ]:
###########################
#   Ex08: Battle_royale   #
###########################

from utils.utils import run_sql


@run_task("Ex08 Battle_royale", oneline=False)
def get_battle_royale():
    res = run_sql("""
                    SELECT Name, Description
                    FROM appstore_games
                    WHERE Description ILIKE '%battle_royale%'
                    AND Description ~
                    '.*https?://(www.)?(facebook|fb).(com).*';
                 """, fetch=True)
    for row in res:
        print(row[0])


def main():
    get_battle_royale()


if __name__ == "__main__":
    main()

In [ ]:
######################
#   Ex09: Benefits   #
######################

from correction.utils.utils import run_sql


@run_task("Ex09 Benefits", oneline=False)
def get_benefits():
    res = run_sql("""
                    SELECT Name
                    FROM appstore_games
                    ORDER BY (User_rating_count * Price) DESC
                    LIMIT 10;
                 """, fetch=True)
    for row in res:
        print(row[0])


def main():
    get_benefits()


if __name__ == "__main__":
    main()

In [ ]:
########################
#   Ex10: Sweet_spot   #
########################

from utils.utils import run_sql


@run_task("Ex10 Sweet_spot", oneline=False)
def get_sweet_spot():
    res = run_sql("""
        SELECT MAX(month)
        FROM (
            SELECT TO_CHAR(Release_date, 'month') AS month, COUNT(*)
            FROM appstore_games
            GROUP BY TO_CHAR(Release_date, 'month')
        ) month_counts
    """, fetch=True)
    for row in res:
        print(row[0])


def main():
    get_sweet_spot()


if __name__ == "__main__":
    main()

In [ ]:
###################
#   Ex11: Price   #
###################


import numpy as np
import matplotlib.pyplot as plt
from utils.utils import run_sql

@run_task("Ex11 Price", oneline=False)
def get_price():
    res= run_sql("""
                    SELECT Price, COUNT(Price)
                    FROM appstore_games
                    WHERE Price > 1
                    GROUP BY Price
                    ORDER BY Price
                 """, fetch=True)
    d = []
    for k, v in res:
        d += [k] * v
    print("mean price : ", np.mean(d))
    print("std price : ", np.std(d))
    bins = [i for i in range(0, 200, 3)]
    plt.hist([e[1] for e in res], bins)
    plt.xlabel("Price")
    plt.ylabel("Frequency")
    plt.title("Appstore games price")
    plt.savefig("price.png")
    plt.show()


def main():
    get_price()


if __name__ == "__main__":
    main()

In [ ]:
#######################
#   Ex12: Worldwide   #
#######################

from utils.utils import run_sql


@run_task("Ex12 Worldwide", oneline=False)
def get_worldwide():
    res = run_sql("""
                SELECT Genre, COUNT(appstore_games_genres.Game_Id)
                FROM appstore_games_genres
                INNER JOIN (
                    SELECT Game_Id, COUNT(Language)
                    FROM appstore_games_languages
                    GROUP BY Game_Id
                    HAVING COUNT(Language) >= 3
                ) Game_lang_freq
                ON appstore_games_genres.Game_Id = Game_lang_freq.Game_Id
                GROUP BY Genre
                ORDER BY COUNT(appstore_games_genres.Game_Id) DESC
                LIMIT 5
                """, fetch=True)
    for e in res:
        print(e[0])


def main():
    get_worldwide()


if __name__ == "__main__":
    main()

In [7]:
####################
#   Ex13: Sample   #
####################

import math
from utils.utils import run_sql


@run_task("Get sample size")
def get_sample_size(
        population,
        confidence=.95,
        error_margin=.05,
        std=0.5):
    conf_zscore = {
        .80 : 1.28,
        .85 : 1.44,
        .90 : 1.65,
        .95 : 1.96,
        .99 : 2.58
    }
    z = conf_zscore[confidence]
    p = std
    num = ((z ** 2) * (p * (1 - p))) / (error_margin ** 2)
    den = 1 + (((z ** 2) * (p * (1 - p))) / ((error_margin ** 2) * population))
    return(math.ceil(num / den))

@run_task("Ex13 Sample", oneline=False)
def sample():
    res = run_sql("""
                SELECT *
                FROM appstore_games
                """, fetch=True)
    sample_size = get_sample_size(len(res))
    conn = get_connection()
    df = pd.read_sql_query("""
                SELECT *
                FROM appstore_games
                ORDER BY random()
                LIMIT {}
                """.format(sample_size), conn)
    df.to_csv("appstore_games.sample.csv", index=False)


def main():
    sample()


if __name__ == "__main__":
    main()

2020-02-22 15:18:47 :: INFO :: Ex13 Sample started.
2020-02-22 15:18:47 :: INFO :: Get sample size Done !
2020-02-22 15:18:47 :: INFO :: Ex13 Sample Done !
